## notebook to check what NMME GCMs have what hindcasts and forecasts on the IRI-DL 

base_url = 'http://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/'

In [309]:
%matplotlib inline
from matplotlib import pyplot as plt

In [310]:
import os 
import sys 
import pathlib

In [311]:
from datetime import datetime, timedelta
import calendar
from dateutil import relativedelta

In [312]:
import numpy as np 
import pandas as pd 
import xarray as xr

In [313]:
CWD = pathlib.Path.cwd()

In [314]:
HOME = pathlib.Path.home()

In [315]:
opath = HOME / 'drives' / 'auck_projects' / 'END19101' / 'Working' / 'data' / 'hindcasts' / 'IRI' / 'CFSv2'

In [316]:
# GCM = 'CanCM4i'
# GCM = 'CMC2-CanCM4'
# GCM = 'CMC1-CanCM3'
# GCM = 'GEM-NEMO'
# GCM = 'NASA-GEOSS2S'
# GCM = 'NCAR-CESM1'
GCM = 'NCEP-CFSv2'

In [318]:
data_stream = 'HINDCAST'
# data_stream = 'FORECAST'

In [319]:
# var = 'Z500'
var = 'T2M'

In [320]:
if var == 'Z500': 
#     url = f'http://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.{GCM}/.{data_stream}/.MONTHLY/.hgt/dods'
    url = f'http://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.{GCM}/.{data_stream}/.MONTHLY/.h500/dods'
elif var == 'T2M': 
    url = f'http://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.{GCM}/.{data_stream}/.MONTHLY/.tref/dods'

### for CFSv2, need to select `.EARLY_MONTH_SAMPLES` as well in the URl 

In [321]:
if GCM == 'NCEP-CFSv2': 

    if data_stream == 'HINDCASTS': 
        url = f'http://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.{GCM}/.{data_stream}/.PENTAD_SAMPLES/.MONTHLY/.tref/dods'
    elif data_stream == 'FORECASTS': 
        url = f'http://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.{GCM}/.{data_stream}/.PENTAD_SAMPLES/.MONTHLY/.tref/dods'

In [322]:
print(url)

http://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.NCEP-CFSv2/.HINDCAST/.MONTHLY/.tref/dods


In [323]:
dset = xr.open_dataset(url, decode_times=False)

In [324]:
dict_coords = {}
dict_coords['L'] = 'step'
dict_coords['M'] = 'member'
dict_coords['S'] = 'time'
dict_coords['X'] = 'lon'
dict_coords['Y'] = 'lat'

In [325]:
dset = dset.rename(dict_coords)

In [326]:
dset

<xarray.Dataset>
Dimensions:  (lat: 181, lon: 360, member: 24, step: 10, time: 348)
Coordinates:
  * lon      (lon) float32 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * step     (step) float32 0.5 1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5
  * member   (member) float32 1.0 2.0 3.0 4.0 5.0 ... 20.0 21.0 22.0 23.0 24.0
  * time     (time) float32 264.0 265.0 266.0 267.0 ... 608.0 609.0 610.0 611.0
  * lat      (lat) float32 -90.0 -89.0 -88.0 -87.0 -86.0 ... 87.0 88.0 89.0 90.0
Data variables:
    tref     (time, step, member, lat, lon) float32 ...
Attributes:
    Conventions:  IRIDL

In [327]:
dset.time

<xarray.DataArray 'time' (time: 348)>
array([264., 265., 266., ..., 609., 610., 611.], dtype=float32)
Coordinates:
  * time     (time) float32 264.0 265.0 266.0 267.0 ... 608.0 609.0 610.0 611.0
Attributes:
    standard_name:  forecast_reference_time
    long_name:      Forecast Start Time
    pointwidth:     0
    calendar:       360
    gridtype:       0
    units:          months since 1960-01-01

### build  the dates 

In [328]:
origin = datetime(1960,1,1)

In [329]:
dates = [origin + relativedelta.relativedelta(months=x) for x in dset.time]

In [330]:
dset['time'] = (('time'), dates)

In [331]:
dset.time[0]

<xarray.DataArray 'time' ()>
array('1982-01-01T00:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 1982-01-01

In [332]:
dset.time[-1]

<xarray.DataArray 'time' ()>
array('2010-12-01T00:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 2010-12-01

In [ ]:
dset = dset.sel(time=slice('1998-10-01',None))

In [ ]:
for date in dset.time: 
    sub = dset.sel(time=date)
    date = pd.to_datetime(date.data)
    print("saving {} for {:%Y-%m}".format(var, date))
    sub.to_netcdf(opath / 'CFSv2_hindcasts_{}_{}_{}.nc'.format(var, date.year, date.month))
    sub.close() 

### alternative using NCKS 

```
ncks -d S,264.,264. http://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.NCEP-CFSv2/.HINDCAST/.MONTHLY/.prec/dods ess.nc
```